In [1]:
%pwd

'd:\\ML_Proj\\End-to-End-ML-Project-with-MLFlow\\research'

In [2]:
import os 

os.chdir("../")

In [3]:
%pwd

'd:\\ML_Proj\\End-to-End-ML-Project-with-MLFlow'

In [4]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class ModelTrainerConfig:
    root_dir: Path
    train_data_path: Path
    test_data_path: Path
    model_name: str
    l1_ratio: float
    alpha: float
    target_column: str


In [5]:
from mlProject.constants import *
from mlProject.utils.common import read_yaml, create_directories

class ConfigurationManager:
    def __init__ (self, 
        config_filepath= CONFIG_FILE_PATH,
        params_filepath= PARAMS_FILE_PATH,
        schema_filepath= SCHEMA_FILE_PATH):

        self.config= read_yaml(config_filepath)
        self.params= read_yaml(params_filepath)
        self.schema= read_yaml(schema_filepath)

        create_directories([self.config.artifacts_root])

    def get_model_trainer_config(self) -> ModelTrainerConfig:
        config= self.config.model_trainer
        schema= self.schema.TARGET_COLUMN
        params= self.params.ElasticNet

        create_directories([config.root_dir])

        model_trainer_config= ModelTrainerConfig(
            root_dir= config.root_dir,
            train_data_path= config.train_data_path,
            test_data_path= config.test_data_path,
            model_name= config.model_name,
            l1_ratio= params.l1_ratio,
            alpha= params.alpha,
            target_column= schema.name
        )
        return model_trainer_config

In [6]:
import os
import joblib
import pandas as pd
from sklearn.linear_model import ElasticNet
from mlProject import logger

class ModelTrainer:
    def __init__ (self, config: ModelTrainerConfig):
        self.config = config

    def train(self):
        train_data= pd.read_csv(self.config.train_data_path)
        test_data= pd.read_csv(self.config.test_data_path)

        train_x= train_data.drop([self.config.target_column], axis=1) 
        test_x= test_data.drop([self.config.target_column], axis=1)
        train_y= train_data[[self.config.target_column]]
        test_y= test_data[[self.config.target_column]]

        lr= ElasticNet(l1_ratio= self.config.l1_ratio, alpha= self.config.alpha, random_state=42)
        lr.fit(train_x, train_y)

        joblib.dump(lr, os.path.join(self.config.root_dir, self.config.model_name))

[2023-08-16 14:33:13,560: INFO: utils: NumExpr defaulting to 8 threads.]


In [7]:
try:
    config= ConfigurationManager()
    model_trainer_config= config.get_model_trainer_config()
    model_trainer= ModelTrainer(config= model_trainer_config)
    model_trainer.train()
except Exception as e:
    raise e

[2023-08-16 14:33:15,670: INFO: common: yaml file: config\config.yaml loaded successfully]
[2023-08-16 14:33:15,672: INFO: common: yaml file: params.yaml loaded successfully]
[2023-08-16 14:33:15,676: INFO: common: yaml file: schema.yaml loaded successfully]
[2023-08-16 14:33:15,677: INFO: common: created directory at: artifacts]
[2023-08-16 14:33:15,678: INFO: common: created directory at: artifacts/model_trainer]
